In [6]:
import json
import logging
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import trange, tqdm
from time import sleep
import pymongo
import requests


In [10]:
client_db = pymongo.MongoClient(port=27017)
db = client_db.py_book
table = db['shahreketab']
df = pd.DataFrame(table.find({}, {
    # '_id': 0
}))
# clean df
df = df[df.url_result != "https://shahreketabonline.com/"]
df = df[df.status_code == 200]
df


,_id,id,url,url_result,text,status_code,cat,title,main_tag
0,657e061660b967f0d3e8a2b7,394634.0,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200.0,کتاب,آرتور و جورج,"<main class=""pb-3"">\n<div class=""container"">\n..."
1,657e061a60b967f0d3e8a2b8,394633.0,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200.0,کتاب,دریاچه ی ارومیه,"<main class=""pb-3"">\n<div class=""container"">\n..."
2,657e061d60b967f0d3e8a2b9,394632.0,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200.0,کتاب,عملیات در تهران (نقشه ی ترور),"<main class=""pb-3"">\n<div class=""container"">\n..."
3,657e062060b967f0d3e8a2ba,394631.0,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200.0,کتاب,موزاییک استعاره ها گفت و گو با بهرام بیضایی,"<main class=""pb-3"">\n<div class=""container"">\n..."
4,657e062360b967f0d3e8a2bb,394630.0,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200.0,کتاب,مدیریت منابع انسانی دیجیتال,"<main class=""pb-3"">\n<div class=""container"">\n..."
...,...,...,...,...,...,...,...,...,...
23879,6586b45ea350b0a28187483a,370754.0,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200.0,NaN,NaN,NaN
23880,6586b461a350b0a28187483b,370753.0,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200.0,NaN,NaN,NaN
23881,6586b464a350b0a28187483c,370752.0,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200.0,NaN,NaN,NaN
23882,6586b467a350b0a28187483d,370751.0,https://shahreketabonline.com/Products/Details...,https://shahreketabonline.com/Products/Details...,"<!DOCTYPE html>\r\n<html dir=""rtl"" lang=""fa"">\...",200.0,NaN,NaN,NaN


In [3]:
# test
# text = df.text[0]
# main_tag = df.main_tag[0]
# soup_all = BeautifulSoup(text)
# soup_main = BeautifulSoup(text)
# cat = soup_main.select('ol.breadcrumb')[0].findAll("li", {"class": "breadcrumb-item"})[1]["title"]
# title = soup_main.select('h1.ProductTitle')[0].text.strip()
# main_tag = soup_main.select('main')[0].__str__()
# shabek = soup_main.select(".Attributes")[0].select( string="ویژگی‌های محصول:")
# shabek

In [8]:
# main_tag
if 'main_tag' not in df.columns:
    df['main_tag'] = None
df_add_field = df[df.main_tag.isnull()]
# df_add_field = df
for i, row in tqdm(df_add_field.iterrows(), total=df_add_field.shape[0]):
    soup = BeautifulSoup(row.text)
    main_tag = soup.select('main')[0].__str__()
    table.update_one({"_id": row["_id"]}, {"$set": {"main_tag": main_tag}})


  0%|          | 0/2656 [00:00<?, ?it/s]

In [11]:
# Cat Calculate
if 'cat' not in df.columns:
    df['cat'] = None
df_add_field = df[df.main_tag.notnull()]
df_add_field = df[df.cat.isnull()]
for i, row in tqdm(df_add_field.iterrows(), total=df_add_field.shape[0]):
    soup = BeautifulSoup(row.main_tag)
    cat = soup.select('ol.breadcrumb')[0].findAll("li", {"class": "breadcrumb-item"})[1]["title"].strip()
    table.update_one({"_id": row["_id"]}, {"$set": {"cat": cat}})

  0%|          | 0/2690 [00:00<?, ?it/s]

TypeError: object of type 'float' has no len()

In [ ]:
# Title Calculate
if 'title' not in df.columns:
    df['title'] = None
df_add_field = df[df.main_tag.notnull()]
df_add_field = df[df.cat.notnull()]
df_add_field = df_add_field[df.title.isnull()]
for i, row in tqdm(df_add_field.iterrows(), total=df_add_field.shape[0]):
    soup = BeautifulSoup(row.main_tag)
    title = soup.select('h1.ProductTitle')[0].text.strip()
    table.update_one({"_id": row["_id"]}, {"$set": {"title": title}})

In [ ]:
# # Shabek Calculate
# if 'shabek' not in df.columns:
#     df['shabek'] = None
# df_add_field = df[df.main_tag.notnull()]
# df_add_field = df[df.cat.notnull()]
# df_add_field = df_add_field[df.shabek.isnull()]
# for i, row in tqdm(df_add_field.iterrows(), total=df_add_field.shape[0]):
#     soup = BeautifulSoup(row.main_tag)
#     title = soup.select('h1.ProductTitle')[0].text.strip()
#     table.update_one({"_id": row["_id"]}, {"$set": {"title": title}})

In [ ]:
df[df['title'].isnull()]

In [ ]:
df[['url', 'url_result', 'status_code', 'cat', 'title']]

In [9]:
df["cat"].value_counts()

cat
کتاب              19669
محصولات فرهنگی      735
صنایع دستی          483
نوشت افزار          208
نشریات               71
کارت هدیه             3
سایر محصولات          2
Name: count, dtype: int64